In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
import cv2

In [2]:
# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
#     gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(img)
    
    if faces == ():
        return None
    
    # Crop all faces found
#     for (x,y,w,h) in faces:
#         cropped_face = img[y:y+h, x:x+w]

    return faces

In [3]:
model=load_model("MaskModel.h5")


In [11]:
cap=cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))
while True:

    ret,frame_org = cap.read()
    if not ret:
        print('error retrieving frame')
        continue
    faces = face_extractor(frame_org)
    
    try:
        for x,y,w,h in faces:
            xn,yn,wn,hn=int(x*1.1),int(y*1.1),int(w*1.3),int(h*1.3) #scale cropped image to better detect mask by model
            cropped_face = frame_org[yn:yn+hn, xn:xn+wn]
            cropped_face=cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB)
            cropped_face=cv2.resize(cropped_face, (200, 200),interpolation= cv2.INTER_LINEAR).reshape(1,200,200,3)
            cropped_face=preprocess_input(cropped_face)
            
            yes,no = model.predict(cropped_face)[0]
            
            if yes>=.5:
                cv2.rectangle(frame_org,(x,y),(x+w,y+h),(0,255,0),2)
                cv2.rectangle(frame_org,(x,y-40),(x+w,y),(0,255,0),-1)
                cv2.putText(frame_org, 'Mask ({}%)'.format(round(yes*100,2)) , (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
                
                
            else:
                cv2.rectangle(frame_org,(x,y),(x+w,y+h),(0,0,255),2)
                cv2.rectangle(frame_org,(x,y-40),(x+w,y),(0,0,255),-1)
                cv2.putText(frame_org, 'No Mask ({}%)'.format(round(no*100,2)) , (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    except:
        pass
        
    out.write(frame_org)   
    cv2.imshow('Frame',   frame_org)
    key = cv2.waitKey(30)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
cap.release()
out.release() 
# Close all started windows
cv2.destroyAllWindows() 


C:\Users\amir\AppData\Local\Temp/ipykernel_5820/184631142.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if faces == ():
